In [27]:
def computeWDS(file_name):
    import numpy as np
    
    index = [0]
    def setGame(mat, strat, N, k, u, tup):
        if k == 0:
            for i in range(0,N):
                mat[tup+(i,)] = u[index[0]]
                #print(u[index[0]])
                index[0] += 1
            return
        
        for i in range(0,strat[k-1]):
            setGame(mat, strat, N, k-1, u, (i,) + tup)
    
    
    f = open(file_name, "r")
    line1 = f.readline()
    line2 = f.readline()
    line3 = f.readline()
    
    # line 2 is important since it contains the number of players and their respective startegies
    data = line2.split(" ")
    data = data[data.index("{") + 1: data.index("}\n")]
    data = data[data.index("{") + 1:]
    num_players = len(data)
    strategies = list(map(int, data))
    
    utilities = line3.split(" ")
    for i in range(0,len(utilities)):
        utilities[i] = float(eval(utilities[i]))
    
    utilities = np.array(utilities)
    
    matrix = np.zeros(tuple(strategies)+(num_players,))
    
    setGame(matrix, tuple(strategies), num_players, num_players, utilities, ())
    #print(matrix)
    # Parsing of .nfg file ends here

    psneTup = []
    def addPsne(tup, strat, N, psneTup):
        psneTup.append(tup)

    def checkPsne(mat, strat, tup, p, util):
        k = 0
        for i in range(0,strat[p]):
            tup = list(tup)
            tup[p] = i
            tup = tuple(tup)
            if util >= mat[tup]:
                k += 1
        if k == strat[p]:
            return 1
        else:
            return 0
    
    def setPsne(mat, N, strat, k, tup):
        if k == 0:
            count = 0
            for i in range(0,N):
                count += checkPsne(mat, strat, tup+(i,), i, mat[tup+(i,)])
            
            if count == N:
                addPsne(tup, strat, N, psneTup)
                
            return
        
        for i in range(0,strat[N-k]):
            setPsne(mat, N, strat, k-1, tup+(i,))
    
    
    def isWeakDominant(mat, N, strat, p1, p2, p3, tup, arr1, arr2):
        if p1==0 and p3==N-1:
            fin11 = True
            fin12 = False
            for i in range(0, strat[p2]):
                if i == tup[p2]:
                    continue
                l = list(tup)
                l[p2] = i
                tup1 = tuple(l)
                fin11 = fin11 and (mat[tup+(p2,)]>=mat[tup1+(p2,)])
                fin12 = fin12 or (mat[tup+(p2,)]>mat[tup1+(p2,)])
            arr1.append(fin11)
            arr2.append(fin12)

        if p1>0:
            for i in range(0, strat[p1-1]):
                isWeakDominant(mat, N, strat, p1-1, p2, p3, (i,)+tup, arr1, arr2)
        if p3<N-1:
            for j in range(0, strat[p3+1]):
                isWeakDominant(mat, N, strat, p1, p2, p3+1, tup+(j,), arr1, arr2)

        fin21 = True
        fin22 = False
        for i in arr1:
            fin21 = fin21 and i
        for i in arr2:
            fin22 = fin22 or i
            
        return fin21 and fin22
        
    domStrats = [-1]*num_players
    def addDom(mat, N, strat, psneTup, domStrats):
        count = 0;
        for i in psneTup:
            count = 0
            for j in i:
                if domStrats[count] == -1 and isWeakDominant(mat, N,strat, count, count, count, (j,), [True], [True]):
                    domStrats[count] = j
                count += 1
        
        makeStrats = []
        for i in range(0, N):
            for j in range(0, strat[i]):
                if j == domStrats[i]:
                    makeStrats.append(1)
                else:
                    makeStrats.append(0)
        return makeStrats
    
    
    setPsne(matrix, num_players, strategies, num_players, ())
    return addDom(matrix, num_players, strategies, psneTup, domStrats)




[0, 0, 0, 0, 0, 0]